# Usuarios y títulos

In [1]:
import csv
import itertools
import os
import re
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

## Socios activos 2016

In [2]:
%%time

dtypes = {
    'CODBAR': str,
    'Sucursal': str,
    'Tratamiento': str,
    'Barrio': str,
    'Ciudad': str,
    'CATEG': str,
    'Ocupación1': str,
    'Nivel Estudio1': str,
    'Área de Estudio1': str,
    'Fech. Nacim': pd.tslib.Timestamp,
    'Edad': float,
}

perfiles_df = pd.read_csv(
    'socios_activos_2016_ciudades_barrios.csv', header=0, dtype=dtypes, parse_dates=['Fech. Nacim'])
perfiles_df = perfiles_df.fillna('')

CPU times: user 395 ms, sys: 11.9 ms, total: 407 ms
Wall time: 406 ms


In [3]:
perfiles_df.shape

(50272, 11)

In [4]:
perfiles_df.head()

,CODBAR,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Fech. Nacim,Edad
0,88180782,BLAA,,chapinero,bogota,CIN,universidad,,,,
1,88183608,BLAA,sra.,ayacuyo ii,bogota,CEM,pensionado,,,1957-07-30 00:00:00,59
2,88181732,BLAA,sra.,icata,bogota,CEM,empleado,profesional,filosofía y letras,,
3,88182347,ARM,sr.,montenegro,montenegro,CEM,empleado,profesional,economía,1963-09-20 00:00:00,53
4,88181284,BLAA,sr.,soledad,bogota,CEM,,,,,


## Columnas útiles

In [5]:
perfiles_cols = [
    'CODBAR',
    'Sucursal',
    'Tratamiento',
#     'Barrio',
#     'Ciudad',
#     'CATEG',
#     'Ocupación1',
#     'Nivel Estudio1',
#     'Área de Estudio1',
#     'Fech. Nacim',
    'Edad',
]

In [6]:
perfiles_df = perfiles_df[perfiles_cols]

In [7]:
perfiles_df.shape

(50272, 4)

## Valores en blanco

#### Sucursal

In [8]:
perfiles_df.Sucursal.value_counts()

BLAA      24278
TUN        3165
IBA        2339
PAS        2089
POP        2078
NEI        1978
PER        1922
MAN        1159
GIR        1090
IPI        1041
CAR         758
SAN         728
CASAGO      686
VAL         660
FLO         611
ARM         575
HON         544
LET         526
BUC         511
QUI         478
STM         438
SIN         438
BUE         337
BAR         328
VIL         298
CAL         287
RIO         250
CUC         236
MED         219
MON         209
             16
Name: Sucursal, dtype: int64

#### Tratamiento

In [9]:
perfiles_df.Tratamiento.value_counts()

sr.          22505
sra.         22341
              4995
otra inf.      392
inst            39
Name: Tratamiento, dtype: int64

#### Edad

In [10]:
perfiles_df.Edad.value_counts()

        6626
22.0    1928
21.0    1860
23.0    1819
20.0    1770
24.0    1653
19.0    1539
17.0    1539
18.0    1463
25.0    1459
16.0    1320
26.0    1235
27.0    1124
28.0    1068
10.0    1001
9.0      993
15.0     989
11.0     960
30.0     934
29.0     927
8.0      887
12.0     817
32.0     794
31.0     793
13.0     766
33.0     696
7.0      694
14.0     692
35.0     677
34.0     647
        ... 
69.0      94
71.0      86
70.0      84
72.0      79
74.0      58
73.0      53
76.0      42
75.0      40
78.0      32
77.0      29
81.0      14
80.0      11
84.0      11
79.0      10
83.0       9
82.0       8
86.0       7
89.0       5
88.0       4
87.0       3
93.0       2
85.0       2
90.0       1
91.0       1
92.0       1
94.0       1
95.0       1
97.0       1
98.0       1
99.0       1
Name: Edad, dtype: int64

## Sucursales

In [11]:
sucursal_ciudad_dict = {
    'ARM': 'Armenia',
    'BAR': 'Barranquilla',
    'BLAA': 'Bogotá',
    'BUE': 'Buenaventura',
    'BUC': 'Bucaramanga',
    'CAL': 'Cali',
    'CAR': 'Cartagena',
    'CASAGO': 'Bogotá',
    'CATMAN': 'Bogotá',
    'CUC': 'Cúcuta',
    'FLO': 'Florencia',
    'GIR': 'Girardot',
    'HON': 'Honda',
    'IBA': 'Ibagué',
    'IEC': 'Bogotá',
    'IPI': 'Ipiales',
    'LET': 'Leticia',
    'MAN': 'Manizales',
    'MED': 'Medellín',
    'MON': 'Montería',
    'NEI': 'Neiva',
    'PAS': 'Pasto',
    'PER': 'Pereira',
    'POP': 'Popayán',
    'QUI': 'Quibdó',
    'RIO': 'Riohacha',
    'SAN': 'San Andrés',
    'SIN': 'Sincelejo',
    'STM': 'Santa Marta',
    'TUN': 'Tunja',
    'VAL': 'Valledupar',
    'VIL': 'Villavicencio',
    '': '',
}

In [12]:
%%time

def process_sucursal(x):
    return sucursal_ciudad_dict[x['Sucursal']]

perfiles_df['Ciudad'] = perfiles_df.apply(process_sucursal, axis=1)

CPU times: user 1.44 s, sys: 6.49 ms, total: 1.45 s
Wall time: 1.45 s


## Datos útiles

In [13]:
perfiles_df = perfiles_df[
    (perfiles_df.Ciudad != '') &
    ((perfiles_df.Tratamiento == 'sr.') | (perfiles_df.Tratamiento == 'sra.')) &
    (perfiles_df.Edad != '')
]

In [14]:
perfiles_df.shape

(39172, 5)

In [15]:
perfiles_df.head()

,CODBAR,Sucursal,Tratamiento,Edad,Ciudad
1,88183608,BLAA,sra.,59,Bogotá
3,88182347,ARM,sr.,53,Armenia
6,88183786,BLAA,sr.,49,Bogotá
8,88185891,BLAA,sr.,56,Bogotá
9,88009796,BLAA,sr.,62,Bogotá


## Grupos

#### Rango_edad

In [16]:
%%time

def bins_to_labels(bins):
    labels = []
    labels.append('-{}'.format(bins[1]))
    for i in range(1, len(bins) - 2):
        labels.append('{}-{}'.format(bins[i]+1, bins[i+1]))
    labels.append('+{}'.format(bins[-2]+1))
    return labels

bins = [-1, 17, 29, 39, 49, 64, 999]
labels = bins_to_labels(bins)

def calculate_rango(x):
    pd.cut(pd.Series(x), bins=bins, labels=labels)

perfiles_df['Rango_edad'] = pd.cut(perfiles_df['Edad'], bins=bins, labels=labels)

CPU times: user 66.4 ms, sys: 0 ns, total: 66.4 ms
Wall time: 65.4 ms


In [17]:
perfiles_df['Rango_edad'].value_counts()

18-29    16157
-17      10780
30-39     6240
40-49     2473
50-64     2398
+65       1124
Name: Rango_edad, dtype: int64

In [18]:
perfiles_df['Rango_edad'].nunique()

6

## Resultados finales

In [19]:
perfiles_df.to_csv('usuarios_2016.csv', index=False)